# CLASE 2.6: Árboles de decisión.
---
## Introducción.
En esta sección introduciremos un nuevo tipo de algoritmo de aprendizaje supervisado en la forma de un marco de referencia o *framework* unificado, conocido como **árbol de decisión**. Dicho *framework* se separa aún más de la teoría que hemos desarrollado previamente en el contexto de este tipo de modelos, debido a que, a diferencia de las máquinas de soporte vectorial que estudiamos en la [clase 2.5](https://github.com/rquezadac/udd_data_science_lectures/blob/main/PARTE%20II%20-%20Modelos%20de%20aprendizaje%20supervisado/clase_2_5.ipynb), los árboles de decisión serán el primer tipo de modelo inherentemente no lineal que vamos a estudiar. La razón fundamental de su estudio separado (y dedicado) estriba en dos pilares fundamentales: Su **simplicidad** y su **éxito** en la resolución de problemas tanto de clasificación como de regresión. Además, constituyen la base de muchos otros modelos que también veremos más adelante. Un ejemplo clásico es el **modelo de bosque aleatorizado** (del inglés **random forest**), que estudiaremos en detalle en la clase 2.7.

El **éxito** de este tipo de modelos y, por extensión, de cualquier otro modelo derivado, guarda relación con varios factores esenciales:

- **No son modelos paramétricos**. Pueden, por tanto, modelar relaciones de alta complejidad entre variables de entrada y salida arbitrarias, **sin necesidad de ningún conocimiento específico a priori**.
- **Pueden tratar con conjuntos de datos heterogéneos**, ya sea compuestos de variables numéricas, discretas, categóricas, o un *mix* de todas ellas.
- **Implementan de forma intrínseca un procedimiento de selección de atributos**, haciéndolos una opción muy robusta para la eliminación de ruido generado por variables irrelevantes y/o redundantes (al menos, hasta cierto punto).
- **Son excelentes para conjuntos de entrenamiento con *outliers* o errores en las etiquetas asociadas a cada observación**.
- **Son fácilmente interpretables**, incluso para desarrolladores sin un *background* completo en aspectos estadísticos.

## Modelos con estructura de tipo árbol.
Sea $\mathcal{D} =\left\{ \left( \mathbf{X} ,\mathbf{y} \right) :\mathbf{X} \in \mathbb{R}^{m\times n} \wedge \mathbf{y} \in \mathbb{R}^{m} \right\}$ un conjunto de entrenamiento caracterizado por una matriz de diseño $\mathbf{X}$ y un vector de valores observados $\mathbf{y}$. Sean $\mathcal{X}$ e $\mathcal{Y}$ los espacios de entrada y salida de $\mathcal{D}$; es decir, el conjunto de todos los valores posibles que pueden tomar $\mathbf{X}$ e $\mathbf{y}$, respectivamente. Cuando el espacio de salida de $\mathcal{D}$ está constituido por un número finito de valores, digamos $\mathcal{Y} =\left\{ c_{1},...,c_{k} \right\}$, donde $c_{r}$ es la $r$-ésima clase de interés (de un total de $k$), solemos decir que $\mathcal{D}$ describe un problema de clasificación multinomial. Una forma útil de visualizar un problema de este tipo es pensar que $\mathbf{y}$ define una partición sobre un universo $\Omega$, tal que

$$\Omega =\Omega_{c_{1}} \cup \Omega_{c_{2}} \cup \cdots \cup \Omega_{c_{k}} =\bigcup^{c_{k}}_{s=c_{1}} \Omega_{c_{s}}$$
<p style="text-align: right;">$(6.1)$</p>

donde $\Omega_{c_{r}}$ es el conjunto de instancias de $\mathcal{D}$ para las cuales $\mathbf{y}$ tiene el valor $c_{r}$. Similarmente, si $f:\mathcal{X} \longrightarrow \mathcal{Y}$ es una hipótesis que hace el trabajo de *predecir* la clase asociada a una instancia $\mathbf{x}_{i}$ (para $i=1,...,m$) de $\mathcal{D}$ (donde $\mathbf{x}_{i}$ es la $i$-ésima fila de $\mathcal{X}$), entonces $f$ también puede verse como una partición de $\Omega$, ya que esta permite obtener una estimación $\hat{\mathbf{y}}$ de $\mathbf{y}$. Sin embargo, dicha partición se define en el espacio de entrada $\mathcal{X}$ en vez de hacerlo directamente en $\Omega$, lo que podemos escribir como

$$\mathcal{X} =\mathcal{X}^{f}_{c_{1}} \cup \mathcal{X}^{f}_{c_{2}} \cup \cdots \cup \mathcal{X}^{f}_{c_{k}} =\bigcup^{c_{k}}_{s=c_{1}} \mathcal{X}^{f}_{c_{s}}$$
<p style="text-align: right;">$(6.2)$</p>

Donde $\mathcal{X}_{c_{r}}^{f}$ es el conjunto de instancias $\mathbf{x}_{i}\in \mathcal{X}$ tales que $f(\mathbf{x}_{i})=c_{r}$. Correspondientemente, el proceso de aprendizaje de un *predictor* $f$ puede replantearse como el aprendizaje de una partición de $\mathcal{X}$ que *mejor coincida* con la partición original generada por $\mathbf{y}$ (siendo ésta última la *partición óptima*).

Desde un punto de vista geométrico, el principio básico de los modelos de árbol de decisión es extraordinariamente simple. Consiste en aproximarse a la partición óptima mediante una secuencia de particiones de $\mathcal{D}$ que, a su vez, produce una colección de subespacios vectoriales, a partir de las cuales se asignan valores constantes como imágenes del predictor $f$ para todas las instancias que viven en cada uno de dichos subespacios.

A fin de ir dando forma de manera más rigurosa a estos conceptos, vamos a formalizar algunas definiciones.

**<font color='blue'>Definición 6.1 – Árbol:</font>** Sea $G=(V,E)$ una red o grafo, donde $V$ es el conjunto de vértices o *nodos* y $E$ es el conjunto de arcos o *caminos*. Asumamos la siguiente terminología: Si $G$ contiene $p$ nodos, entonces $V$ puede expresarse por medio del conjunto $V=\left\{ 1,...,p\right\}$, siendo entonces $E=\left\{ \left( s,t\right)  :s\wedge t\in V,s\neq t\right\}$ el conjunto de todos los arcos de la red que unen a los nodos $s$ y $t$. Un **árbol** se define como un grafo $G=(V,E)$ tal que cualquier combinación de nodos está conectada por uno y sólo un arco.

Al respecto, es posible observar varios aspectos importantes en una red de tipo árbol:

- **(T1):** Si $G=(V,E)$ representa a un árbol, entonces es común designar a uno de sus nodos como la **raíz** del mismo. De ser así, entonces necesariamente $G$ será un **grafo dirigido** (es decir, los arcos $E$ tendrán **direcciones restringidas** –por ejemplo, $(s,t)$ podría recorrerse desde $s$ hasta $t$, pero no al revés–) para el cual todos los arcos se *alejarán* de dicho nodo raíz. Un árbol así definido es llamado **árbol con raíz**.
- **(T2):** Si existe un arco que va desde $s$ a $t$ (es decir, $(s,t)\in E$), entonces diremos que $s$ es el **nodo padre** (o, simplemente, *padre*) del nodo $t$. Correspondientemente, el nodo $t$ será llamado **nodo hijo** (o simplemente *hijo*) del nodo $s$.
- **(T3):** Para un árbol con raíz, diremos que un nodo es **interno** si tiene uno o más hijos, y **términal** si no los tiene. Los nodos terminales, sobretodo en la teoría de aprendizaje automatizado, suelen ser llamados **nodos hoja**, o simplemente **hojas**.
- **(T4):** Un **árbol binario** es un árbol con raíz para el cual cada nodo interno tiene siempre dos hijos.

En estos términos, un **modelo con estructura de tipo árbol** o, derechamente, **árbol de decisión**, puede definirse como un objeto matemático $f:\mathcal{X}\longrightarrow \mathcal{Y}$, representado por un árbol con raíz (con frecuencia, binario, aunque no necesariamente tiene que ser así), donde cualquier nodo $t$ representa un subespacio $\mathcal{X}_{t}\subset \mathcal{X}$ del espacio de entrada y el nodo raíz, que llamamos $t_{0}$ corresponde a todo el conjunto $\mathcal{X}$. Los nodos internos del árbol se etiquetan con una **separación** o **split** $s_{t}$ que se construye a partir de una serie de *"preguntas"* que constituyen un conjunto $\mathcal{P}$. Cada *split* divide al subespacio $\mathcal{X}_{t}$ que representa al nodo $t$ en un número de subespacios disjuntos, cada uno de los cuales se corresponde con los nodos hijos de $t$. Por ejemplo, el conjunto de todos los *splits* binarios equivale a un conjunto $\mathcal{P}$ de preguntas $s$ de la forma: *"¿La instancia $\mathbf{x}_{i}$ pertence a $\mathcal{X}_{r}$?"*, donde $\mathcal{X}_{r}\subset \mathcal{X}$ es *algún* subconjunto del espacio de entrada. De esta manera, cualquier *split* $s$ divide a $\mathcal{X}_{t}$ en dos subespacios respectivamente. Si desginamos por $t$ a este nodo, entonces tales subespacios serán $\mathcal{X}_{t}\cap \mathcal{X}_{r}$ para el nodo hijo hacia la *izquierda* de $t$, y $\mathcal{X}_{t} \cap (\mathcal{X}\setminus  \mathcal{X}_{r})$ para el nodo hijo hacia la *derecha* de $t$. Los nodos terminales se etiquetan con las *mejores* estimaciones $\hat{y}_{t}\in \mathcal{Y}$ para la variable de salida u objetivo. Si $f$ representa a un **árbol de clasificación**, entonces $\hat{y}_{t} \in \left\{ c_{1},\cdots ,c_{k}\right\}$, mientras que, si $f$ es un **árbol de regresión**, entonces $\hat{y}\in [a,b]$, donde $[a,b]$ es un intervalo cerrado en $\mathbb{R}$. Bajo estas condiciones, la predicción $f(\mathbf{x}_{i})$ es el valor con el cual se etiqueta la hoja a la cual se llega mediante la instancia $\mathbf{x}_{i}$, al propagarse por el árbol siguiendo los *splits* $s_{t}$, lo que puede estructurarse conforme el algoritmo (6.1). Notemos que dicho algoritmo se escribe en *pseudo-código*.

<p style="text-align: center;">Algoritmo (6.1): Ejemplo básico de algoritmo que predice la salida $\hat{y}=f(\mathbf{x}_{i})$ en un árbol de decisión para una instacia $\mathbf{x}_{i}$</p>

**`def`** `predict`$(f,\mathbf{x}_{i})$<br>
>$t=t_{0}$<br>
**`while`** $t$ no sea un nodo terminal:<br>
>>$t=$el nodo hijo $t^{\ast}$ de $t$ tal que $\mathbf{x}_{i}\in \mathcal{X}_{t^{\ast}}$<br>

>**`return`** $\hat{y}_{t}$

La Fig. (6.1) ilustra un modelo de árbol de decisión $f$ compuesto por cinco nodos y que particiona el espacio de entrada $\mathcal{X}=\mathcal{X}_{1}\times \mathcal{X}_{2}=[0,1]\times [0,1]$ para un problema de clasificación binaria (donde el espacio de salida es $\mathcal{Y}=\left\{ c_{1},c_{2}\right\}$). El nodo $t_{0}$ es la raíz del árbol y corresponde al espacio de entrada completo; es decir, $\mathcal{X}_{t_{0}}=\mathcal{X}$. Dicho nodo se etiqueta con el *split* binario 

$$s_{0}=\left\{ x_{i1}\leq 0.7\right\}$$
<p style="text-align: right;">$(6.3)$</p>

Es decir, dicho *split* formula la siguiente pregunta: *"¿Es $x_{i1}$ menor o igual que $0.7$?"*, donde $x_{i1}$ hace referencia a la $i$-ésima instancia (o fila) con respecto a la primera ($j=1$) variable independiente (o columna) asociada a la matriz de diseño $\mathbf{X}\in \mathbb{R}^{m\times 2} (1\leq i\leq m)$. Es decir, **cada split define una partición con respecto a una de las $2$ variables independientes del conjunto de entrenamiento**. Por el momento, no nos preocuparemos de cómo se escoge la variable asociada al *split*. Simplemente asumiremos que *una de ellas* es la descrita por el *split* y la que particiona a $\mathcal{X}$ en cada uno de los nodos no terminales del árbol.

<p style="text-align: center;"><img src="figures/fig_6_1.png" width="350"></p>
<p style="text-align: center;">Fig. (6.1): Un sencillo árbol de decisión construido para resolver un problema de clasificación a partir de<br>un espacio de entrada definido por el conjunto $\mathcal{X}=[0,1]\times [0,1]$</p>

Aclarada la notación, prosigamos. El split $s_{0}=\left\{ x_{i1}\leq 0.7 \right\}$, asociado al nodo $t_{0}$, divide al espacio $\mathcal{X}_{0}$ (que coincide con el espacio de entrada $\mathcal{X}$) en dos subespacios disjuntos, que denotamos como $\mathcal{X}_{t_{1}} \wedge \mathcal{X}_{t_{2}}$. El subespacio $\mathcal{X}_{t_{1}}$ corresponde al nodo hijo $t_{1}$ ubicado a la izquierda de $t_{0}$ y representa a todas las instancias $x_{i1}\in \mathcal{X}_{t_{0}}$ (para $i=1,...,m$) tales que $x_{i1}\leq 0.7$. De la misma forma, $t_{1}$ se rotula con el split $s_{1}=\left\{ x_{i2}\leq 0.5 \right\}$, que a su vez divide al espacio $\mathcal{X}_{t_{1}}$ en dos subespacios disjuntos, denominados como $\mathcal{X}_{t_{3}}\wedge \mathcal{X}_{t_{4}}$, y que representan –respectivamente– al conjunto de todas las instancias $x_{i2}\in \mathcal{X}_{t_{1}}$ tales que $x_{i2}\leq 0.5$ y $x_{i2}> 0.5$. Los nodos terminales $t_{2},t_{3}$ y $t_{4}$ se representan en el esquema del árbol como *cuadrados* que han sido etiquetados con un valor de salida o **predicción** $\hat{y}_{t}$. En conjunto, estos valores constituyen una **partición** de $\mathcal{X}$ (en el sentido de la ecuación (6.2)), donde cada conjunto $\mathcal{X}_{c_{k}}^{f}$ se obtiene a partir de la unión de los subespacios $\mathcal{X}_{t}$ de todos los nodos terminales $t$ tales que $\hat{y}_{t}=c_{k}$. En este ejemplo particular, $\mathcal{X}_{c_{1}}^{f} =\mathcal{X}_{t_{4}}$, mientras que $\mathcal{X}_{c_{2}}^{f} =\mathcal{X}_{t_{2}} \cup \mathcal{X}_{t_{3}}$.

<p style="text-align: center;"><img src="figures/fig_6_2.png" width="780"></p>
<p style="text-align: center;">Fig. (6.2): Partición de $\mathcal{X}=[0,1]\times [0,1]$ inducida por el árbol de decisión $f$ que divide a $\mathcal{X}$ en subespacios homogéneos. Los puntos rojos corresponden a objetos de la clase $c_{1}$, mientras que los puntos azules corresponden a objetos de la clase $c_{2}$</p>

Como se muestra en la Fig. (6.2), la partición inducida por el árbol de clasificación $f$ divide al espacio de entrada $\mathcal{X}$ en subespacios que son más y más homogéneos con respecto a ambas clases, partiendo desde $\mathcal{X}$ en el nodo raíz, luego $\mathcal{X}_{t_{1}}\cup \mathcal{X}_{t_{2}}$ en el segundo nivel del árbol, y finalmente, $\left( \mathcal{X}_{t_{3}} \cup \mathcal{X}_{t_{4}} \right) \cup \mathcal{X}_{t_{2}}$ en las hojas. Como veremos más adelante, la partición en este caso está constituida por rectángulos debido a la naturaleza de los *splits* $s_{t}\in \mathcal{P}$ que caracterizan a cada nodo del árbol de decisión. Las predicciones se realizan por medio de la propagación de las instancias a través del árbol y usando como valor de salida del modelo a las etiquetas de cada una de sus hojas dependiendo de donde éstas instancias *caigan*, según sea el caso. Por ejemplo, el punto $(x_{1},x_{2})= (0.2, 0.7)$, al propagarse por el árbol ilustrado en la Fig. (6.1), cae en el nodo $t_{4}$, y por lo tanto el modelo produce la estimación $\hat{y}_{t_{4}}=f(0.2, 0.7)=c_{1}$.

## Inducción de árboles de decisión.
Sea $\mathcal{D} =\left\{ \left( \mathbf{X} ,\mathbf{y} \right) :\mathbf{X} \in \mathbb{R}^{m\times n} \wedge \mathbf{y} \in \mathbb{R}^{m} \right\}$ un conjunto de datos de entrenamiento, donde la fila $\mathbf{x}_{i}\in \mathbb{R}^{n}$ hace referencia a una instancia de $\mathbf{X}$ e $y_{i}$ es la correspondiente observación de $\mathbf{y}$. Sea $f:\mathcal{X}\longrightarrow \mathcal{Y}$ un modelo de tipo árbol, con $\mathcal{X}\wedge \mathcal{Y}$ los espacios de entrada y salida de $\mathcal{D}$, respectivamente. El proceso de aprendizaje de un árbol de decisión idealmente apunta a determinar la estructura de tipo árbol que produce la partición más *próxima* o *similar* a la partición generada por $\mathcal{Y}$ sobre $\mathcal{X}$. Debido a que tal estructura es desconocida, la construcción de un árbol de decisión suele realizarse con base en la determinación de un modelo que particione al conjunto de entrenamiento $\mathcal{D}$ *lo mejor posible*. Entre todos los árboles de decisión $f\in \mathcal{H}$ (donde $\mathcal{H}$ es el universo de todos los árboles de decisión posibles), pueden existir varios que expliquen la estructura de $\mathcal{D}$ igualmente bien. Una opción lógicamente válida es preferir aquel modelo que haga el menor número de supuestos posibles; esto es, elegir el modelo más simple de todos los que se ajustan igualmente bien a nuestros datos. Por lo tanto, el aprendizaje de un árbol de decisión $f$ a partir de $\mathcal{D}$ suele definirse como el hallar el árbol más pequeño $f^{\ast}$ (en términos de sus nodos internos) que minimice el error promedio de estimación de $f$ sobre $\mathcal{D}$:

$$E\left( f,\mathcal{D} \right) =\frac{1}{m} \sum_{\left( \mathbf{x}_{i} ,y_{i} \right) \in \mathcal{D}} L\left( y_{i},f\left( \mathbf{x}_{i} \right) \right)$$
<p style="text-align: right;">$(6.4)$</p>

Donde $L$ es una **función de costo** que mide la discrepancia entre las observaciones $y_{i}$ y el modelo $f(\mathbf{x}_{i})$.

La elección del *árbol mínimo* $f^{\ast}$ es un supuesto válido y razonable desde el punto de vista de la generalización del aprendizaje y también desde la perspectiva de la interpretabilidad: Un árbol de decisión pequeño es mucho más fácil de interpretar que uno más grande. Sin embargo, el problema de determinar dicho árbol mínimo es intratable al ser $NP$-completo. Es decir, no existe un algoritmo eficiente para encontrar el árbol $f^{\ast}$ en tiempo polinómico, lo que nos fuerza a construir heurísticas que permitan llegar a un resultado aproximado. La razón de por qué este problema es intratable se desarrollará en el siguiente ejemplo.

**Ejemplo 5.1 – Intratabilidad del problema de encontrar un árbol de decisión mínimo:** Consideremos nuevamente el conjunto de entrenamiento $\mathcal{D}$ descrito más arriba. Cada instancia $\mathbf{x}_{i}$ *vive* en el espacio $\mathcal{X}$. Por simplicidad, de manera similar a como lo hicimos en el primer ejercicio que realizamos, podemos pensar que $\mathcal{X} \subseteq \left\{ 0,1 \right\}^{n}$ (es decir, cada variable independiente de $\mathcal{D}$ es binaria, y toma los valortes $0$ y $1$). Asumiremos que $\mathbf{y}$ está constituido igualmente por valores iguales a $0$ o $1$. 

Un árbol de decisión (binario) válido para $\mathcal{D}$ es tal que sus hojas están etiquetadas igualmente con los números $0$ o $1$ por medio de *splits* (o "preguntas") que caracterizan a cada nodo intermedio. De esta manera, $\mathcal{D}$ representa a un problema de clasificación cuya resolución equivale a encontrar una función Booleana $f$ capaz de asignar los valores $0$ y $1$ a las variables $\mathbf{x}_{j}\in \mathbb{R}^{m}$ ($j=1,...,n$). Equivalentemente, un árbol de decisión que satisface este problema equivale a determinar "alguna" manera de asignar los valores $0$ y $1$ a las instancias $\mathbf{x}_{1},...,\mathbf{x}_{m}$, de manera tal que todos los *splits* que constituyen los nodos intermedios del árbol separen perfectamente a los datos, satisfaciendo en cada caso las clases en cada una de sus hojas. No existe una única forma de resolver este problema y, por extensión, tampoco hay un único árbol que los satisfaga. Por lo tanto, es natural *intentar* determinar, de entre todos ellos, el árbol con la menor cantidad de nodos intermedios (o *splits*), llamado **árbol óptimo o mínimo**. Este problema particular es computacionalmente intratable.

Para demostrarlo, consideremos una tabla de verdad que represente a las variables independientes (Booleanas) $\mathbf{x}_{j} (j=1,...,n)$. El número total de combinaciones de valores binarias entre estas $n$ variables, que representa al número de filas de esta tabla de verdad, es igual a $2^{n}$. Por ejemplo, si $n=3$, se tendrá la tabla de verdad mostrada a continuación:

<p style="text-align: center;">Tabla (6.1): Tabla de verdad para tres variables binarias. Su número de filas es igual a $2^{3}=8$</p>

| Variable $\mathbf{x}_{1}$ | Variable $\mathbf{x}_{2}$ | Variable $\mathbf{x}_{3}$ |
| :--------------: | :---------------: | :---------------: |
| 0 | 0 | 0 |
| 0 | 0 | 1 |
| 0 | 1 | 0 |
| 1 | 0 | 0 |
| 1 | 1 | 0 |
| 1 | 0 | 1 |
| 0 | 1 | 1 |
| 1 | 1 | 1 |

Y que tiene un total de $2^{3}=8$ filas. Para el caso general, si el valor de verdad de cada combinación también es binario, hay $2^{\left( 2^{n} \right)}$ posibles funciones de clasificación. Es decir, existen $2^{\left( 2^{n} \right)}$ formas diferentes de etiquetar cada una de las filas que constituyen las combinaciones en la correspondiente tabla de verdad. Esto significa que la cantidad de formas en las cuales podemos definir un *split* en un árbol de decisión que particione eficazmente al espacio de entrada del problema es, al menos, igual de grande, porque más de un árbol de decisión puede ser igualmente útil para particionar dicho espacio con los mismos resultados (o, palabras más simples, más de un árbol puede ser válido para representar a una de estas funciones de clasificación). Debido a que, de entre todas estas opciones, hay sólo una que nos interesa (la que tiene la menor cantidad de nodos intermedios o *splits*), la resolución del problema que implica encontrar el árbol de decisión mínimo requiere, literalmente, fuerza bruta para recorrer todo el espacio de búsqueda para seleccionar el árbol más pequeño de todos. Cada variable independiente que añadimos en nuestro conjunto de entrenamiento hace crecer el espacio de búsqueda del árbol mínimo de forma *super-exponencial*, lo que hace a este problema intratable (en tiempo *polinomial*).

¿Qué tan grande es el número $2^{\left( 2^{n} \right)}$? Pues bueno... muchísimo. Un problema de clasificación constituido únicamente por $10$ variables binarias tendrá $2^{\left( 2^{10} \right)}=2^{1024}$ formas distintas de etiquetar cada salida. Este número es, aproximadamente, $10^{228}$ veces más grande que la cantidad de átomos existente en el universo observable. Y sólo hablamos de un problema binario, con variables independientes binarias. Para un problema multinomial, o con variables independientes continuas, dicho número crece, para efectos prácticos, de forma infinita. ◼︎

Habiendo establecido lo impracticable que resulta determinar un árbol de decisión mínimo para cualquier problema de aprendizaje supervisado, queda en evidencia que el desarrollo de la teoría de los árboles de decisión debe apuntar hacia procedimientos heurísticos que nos permitan llegar a un resultado aproximado. En primer lugar, vamos a desarrollar una noción de un elemento fundamental en la construcción de tales heurísticas, conocido como **métrica de impureza**, denotada como $i(t)$, y que permite *evaluar* la *bondad* o *idoneidad* de un nodo $t$ en un árbol de decisión. Asumamos que, mientras más pequeño sea $i(t)$, más *puro* será el nodo $t$, resultando en mejores predicciones $\hat{y}_{t}(\mathbf{x}_{i})$ para todo $\mathbf{x}_{i}\in \mathcal{D}_{t}$ ($i=1,...,m$), donde $\mathcal{D}_{t}$ es el subconjunto de instancias en el conjunto de entrenamiento $\mathcal{D}$ que caen en $t$ (esto es, todo par $(\mathbf{x}_{i},y_{i})\in \mathcal{D}$ tal que $\mathbf{x}_{i}\in \mathcal{X}_{t}$, donde $\mathcal{X}_{t}$ es el subespacio que contiene a todas las instancias que se propagan por el nodo $t$). Partiendo desde un nodo raíz que representa a todo el espacio de entrada $\mathcal{X}$, podemos *hacer crecer* árboles casi-óptimos por medio de un *procedimiento codicioso*, dividiendo cada nodo interno en otros nodos que secuencialmente se hacen más y más *puros* de manera iterativa. Esto es, dividiendo iterativamente el espacio de entrada $\mathcal{X}$ en subespacios cada vez más pequeños (representados por los nodos del árbol), hasta que los nodos terminales (hojas) ya no pueden hacerse más *puros*, lo que permite garantizar predicciones *casi óptimas* sobre el conjunto de entrenamiento $\mathcal{D}$. El *supuesto codicioso* para intentar lograr que el árbol resultante sea lo más pequeño posible (lo que permite, en teoría, una buena generalización) es, por tanto, dividir cada nodo $t$ del árbol usando el *split* $s^{\ast}$ que maximiza localmente la caída de impureza de los nodos hijos resultantes. Tiene sentido, por ende, la siguiente definición.

**<font color='blue'>Definición 6.2 – Caída de impureza:</font>** Sea $G(V,E)$ un árbol de decisión y sea $t\in E$ un nodo interno de $G(V,E)$. Sea $s_{t}$ el split (binario) que rotula al nodo $t$, y sean $t_{L}$ y $t_{R}$ los nodos hijos ubicados a la izquierda y derecha de $t$, respectivamente. La **caída de impureza** del split $s_{t}$ se define como

$$\triangle i\left( s_{t},t \right) =i\left( t \right) -p_{L}i\left( t_{L} \right) -p_{R}i\left( t_{R} \right)$$
<p style="text-align: right;">$(6.5)$</p>

Donde:

- $p_{L}$ es la proporción $\frac{m_{t_{L}}}{m_{t}}$ de instancias de entrenamiento de $\mathcal{D}_{t}$ que se propagan por $t$ hacia $t_{L}$, siendo $m_{t}$ la cantidad de instancias de $\mathcal{D}_{t}$.
- $p_{R}$ es la proporción $\frac{m_{t_{R}}}{m_{t}}$ de instancias de entrenamiento de $\mathcal{D}_{t}$ que se propagan por $t$ hacia $t_{R}$.

**Observación:** Notemos que, hasta ahora, hemos utilizado dos notaciones para designar a un árbol de decisión: Una que hace mención a su estructura de tipo grafo ($G(V,E)$), y otra que hace mención a su representatividad como función clasificadora (o regresora, según sea el caso, $f$). Ambas notaciones son equivalentes, y se utilizarán dependiendo de lo que queramos presentar o definir en base al mismo árbol de decisión.

Sobre la base de la definición (6.2), podemos describir formalmente un procedimiento iterativo de tipo *codicioso* que permita generar un árbol de decisión a partir de un conjunto de entrenamiento $\mathcal{D} =\left\{ \left( \mathbf{X} ,\mathbf{y} \right) :\mathbf{X} \in \mathbb{R}^{m\times n} \wedge \mathbf{y} \in \mathbb{R}^{m} \right\}$, donde $\mathbf{X}$ es la matriz de diseño e $\mathbf{y}$ es el vector de valores observados. Tal procedimiento se ilustra en el algoritmo (6.2) en un formato de pseudo-código.



<p style="text-align: center;">Algoritmo (6.2): Inducción codiciosa de un árbol de decisión binario</p>

**`def`** `BuildDecisionTree`$(\mathcal{D})$<br>
>Crear un árbol de decisión $f$ con nodo raíz $t_{0}$<br>
>Crear un contenedor $S$ vacío de nodos *abiertos* $(t,\mathcal{D}_{t})$<br>
>Agregar $(t_{0},\mathcal{D}_{t})$ a $S$<br>
>**`while`** $S$ no esté vacío:<br>
>>$t,\mathcal{D}_{t}=\mathrm{POP}(S)$<br>
>>**`if`** el criterio de detención se cumple para $t$:
>>>$\hat{y}_{t} = \mathrm{algun\ valor\ constante}$<br>

>>**`else`**:
>>>Encontrar el split en $\mathcal{D}_{t}$ que maximiza la caída de impureza: $s^{\ast}=\underset{s_{t}}{\mathrm{argmax}} \left( \triangle i\left( s_{t},t \right) \right)$<br>
>>>Particionar $\mathcal{D}_{t}$ en $\mathcal{D}_{t_{L}}\cup \mathcal{D}_{t_{R}}$, de acuerdo a $s^{\ast}$<br>
>>>Crear el nodo hijo $t_{L}$ a la izquierda de $t$<br>
>>>Crear el nodo hijo $t_{R}$ a la derecha de $t$<br>
>>>Agregar $(t_{L},\mathcal{D}_{t_{L}})$ a $S$<br>
>>>Agregar $(t_{R},\mathcal{D}_{t_{R}})$ a $S$<br>

>**`return`** $f$

En lo que resta de esta sección, nos dedicaremos a explicar en detalle todas las piezas que conforman el algoritmo (6.2). Primero, estudiaremos las **reglas de asignación** para los nodos terminales de un árbol de decisión (que es lo que ocurre cuando se cumple el *criterio de detención*). Luego, definiremos dichos criterios de detención. A continuación, presentaremos familias $\mathcal{P}$ de reglas de separación o *splitting*, criterios de impureza para evaluar la idoneidad de los *splits* y estrategias para encontrar los mejores *splits* $s^{\ast}\in \mathcal{P}$. Como veremos más adelante, la parte esencial del algoritmo (6.2) será el cómo encontrar buenos *splits* y cómo determinar cuándo dejamos de separar los nodos de un árbol.

## Reglas de asignación.
Sea $\mathcal{D} =\left\{ \left( \mathbf{X} ,\mathbf{y} \right) :\mathbf{X} \in \mathbb{R}^{m\times n} \wedge \mathbf{y} \in \mathbb{R}^{m} \right\}$ un conjunto de datos de entrenamiento, donde la fila $\mathbf{x}_{i}\in \mathbb{R}^{n}$ hace referencia a una instancia de $\mathbf{X}$ e $y_{i}$ es la correspondiente observación de $\mathbf{y}$, siendo $\mathcal{X}\wedge \mathcal{Y}$ los correspondientes espacios de entrada y salida de $\mathcal{D}$, respectivamente. Consideremos un árbol de decisión $G(V,E)$ constituido por una secuencia de nodos, uno de los cuales se representa arbitrariamente como $t\in E$. Asumamos que dicho nodo $t$ ha sido declarado como terminal por el algoritmo (6.2) por algún criterio de detención (que, por el momento, no será relevante). El siguiente paso en dicho algoritmo es etiquetar al nodo $t$ con un valor constante, que llamamos $\hat{y}_{t}$, que será utilizado como una predicción para una de las instancias de la variable de salida $\mathbf{y}$. Como tal, $t$ puede considerarse como un modelo muy simple, localmente definido en $\mathcal{X}_{t}\times \mathcal{Y}$, el que produce la misma predicción $\hat{y}_{t}$ para todas las posibles instancias que caen en $t$.

Observamos primero que, para un modelo de tipo árbol $f$ de estructura *fija*, la minimización del error promedio de estimación es estrictamente equivalente a minimizar cada término de error local asociado a los modelos locales que describen los nodos terminales. Es decir,

$$\begin{array}{lll}E\left( f,\mathcal{D} \right)&=&\mathrm{E}_{X,Y} \left[ L\left( \mathbf{y} ,f\left( \mathbf{X} \right) \right) \right]\\ &=&\displaystyle \sum_{t\in \tilde{f}} P\left( X\in \mathcal{X}_{t} \right) \mathrm{E}_{X,Y|t} \left[ L\left( \mathbf{y} ,\hat{y}_{t} \right) \right]\end{array}$$
<p style="text-align: right;">$(6.6)$</p>

Donde:

- $\tilde{f}$ es el conjunto de nodos terminales en $f$.
- $X$ e $Y$ son variables aleatorias respecto de las cuales $\mathbf{X}$ e $\mathbf{y}$ son asumidas como realizaciones.
- El valor esperado conjunto $\mathrm{E}_{X,Y}$ se toma sobre todas las instancias $\mathbf{x}_{i}\in \mathcal{X}_{t}$.

Por lo tanto, sobre la base de la fórmula (6.6), un modelo que minimiza $E\left( f,\mathcal{D} \right)$ es equivalente a un modelo que minimiza el valor esperado de los errores en cada hoja del árbol, ponderando cada término de error por la probabilidad de que una instancia se propague o *caiga* en dicha hoja. De esta manera, el aprendizaje del *mejor* árbol de decisión posible simplemente apunta a encontrar la mejor colección de constantes en cada uno de los correspondientes nodos terminales.

### Problema de clasificación.
Consideremos el caso en el cual el conjunto de entrenamiento $\mathcal{D} =\left\{ \left( \mathbf{X} ,\mathbf{y} \right) :\mathbf{X} \in \mathbb{R}^{m\times n} \wedge \mathbf{y} \in \mathbb{R}^{m} \right\}$ representa a un problema de clasificación. Es decir, el vector $\mathbb{y}\in \mathbb{R}^{m}$ está constituido por un número finito de valores discretos. Sin pérdida de generalidad, consideraremos el caso binario, en el cual $\mathbb{y}$ únicamente toma los valores $y_{i}=0$ o $y_{i}=1$, para todo $i=1,...,m$. Una opción para la función de costo $L$ en un escenario como este corresponde a la función $\mathbf{1}:\mathbb{R} \longrightarrow \left\{ 0,1 \right\}$, denominada **pérdida binaria** o **pérdida de clasificación**, definida como

$$\mathbf{1}\left( y\neq\hat{y} \right) =\begin{cases}0&;\  \mathrm{si} \  y=\hat{y}\\ 1&;\  \mathrm{si} \  y\neq \hat{y}\end{cases}$$
<p style="text-align: right;">$(6.7)$</p>

La función (6.7) es muy sencilla, y suele utilizarse en la construcción de modelos de clasificación con el objetivo de maximizar la exactitud de tales modelos. Sea pues $f$ un modelo de tipo árbol y $y$ un nodo terminal arbitrario de $f$. Denotemos por $\mathcal{X}_{t}$ el subespacio de instancias de $\mathcal{D}$ que se propagan por el árbol hasta $t$, siendo $m_{t}$ la cantidad de instancias en $\mathcal{X}_{t}$. Si reemplazamos la función de costo (6.7) en el argumento del valor esperado condicional interno $\mathrm{E}_{X,Y|t}$ en la última línea de la ecuación (6.6), obtendremos

$$\begin{array}{lll}\hat{y}_{t}^{\ast}&=&\displaystyle \underset{c\in \mathcal{Y}}{\mathrm{argmin}} \left( \mathrm{E}_{X,Y|t} \left[ \mathbf{1} \left( y,c \right) \right] \right)\\ &=&\displaystyle \underset{c\in \mathcal{Y}}{\mathrm{argmin}} \left( P\left( y_{i}\neq c\  |\  \mathbf{x}_{i} \in \mathcal{X}_{t} \right) \right) \  ;\  i=1,...,m\\ &=&\displaystyle \underset{c\in \mathcal{Y}}{\mathrm{argmax}} \left( P\left( y_{i}=c\  |\  \mathbf{x}_{i} \in \mathcal{X}_{t} \right) \right)\end{array}$$
<p style="text-align: right;">$(6.8)$</p>

Donde $c\in \mathbb{R}$ es la constante que el nodo terminal $t$ asigna como salida. Dicho de otro modo, el error de generalización de $t$ se minimiza prediciendo la clase más frecuente (o más probable) en el subespacio de datos que cae en $t$. Si hay más de una clase que minimiza dicho error, basta con escoger cualquiera de ellas como $\hat{y}_{t}^{\ast}$. Este resultado se conoce en la teoría de los árboles de decisión como **regla de pluralidad**.

Para resolver la ecuación (6.8), en la práctica, es necesario conocer la función de densidad conjunta $P_{X,Y}$. Dado que en la mayoría de problemas esto no está disponible explícitamente, es común aproximar esta distribución mediante estimaciones de la distribución local de cada nodo terminal. Sea $p\left( c|t \right) =\frac{n_{c}\left( t \right)}{m_{t}}$ la proporción de las $n_{c}(t)$ instancias que pertenecen a la clase $c$ y que caen en $t$ con respecto al total $m_{t}$ de instancias en el subespacio $\mathcal{X}_{t}$, y que proponemos como una estimación de la probabilidad $P\left( y_{i}=c\  |\  \mathbf{x}_{i} \in \mathcal{X}_{t} \right)$. Reemplazando en la expresión (6.8), obtenemos

$$\begin{array}{lll}\hat{y}_{t}&=&\displaystyle \underset{c\in \mathcal{Y}}{\mathrm{argmin}} \left( 1-p\left( c|t \right) \right)\\ &=&\displaystyle \underset{c\in \mathcal{Y}}{\mathrm{argmax}} \left( p\left( c|t \right) \right)\end{array}$$
<p style="text-align: right;">$(6.9)$</p>

De manera análoga, definimos la proporción $p\left( t \right) =\frac{m_{t}}{m}$ como la estimación de la probabilidad $P(\mathbf{x}_{i}\in \mathcal{X}_{t})$. Reemplazando ambas estimaciones en la expresión (6.6), nos da

$$\begin{array}{lll}\hat{E} \left( f,\mathcal{D} \right)&=&\displaystyle \sum_{t\in \tilde{f}} \underbrace{p\left( t \right)}_{=\frac{m_{t}}{m}} \  \underbrace{\left( 1-p\left( \hat{y}_{t} |t \right) \right)}_{\mathrm{regla\  de\  pluralidad}}\\ &=&\displaystyle \sum_{t\in \tilde{f}} \frac{m_{t}}{m} \left( 1-\frac{n_{c}\left( t \right)}{m_{t}} \right)\\ &=&\displaystyle \frac{1}{m} \sum_{t\in \tilde{f}} \left( m_{t}-n_{c}\left( t \right) \right)\\ &=&\displaystyle \frac{1}{m} \sum_{t\in \tilde{f}} \sum_{\left( \mathbf{x}_{i} ,y_{i} \right) \in \mathcal{D}_{t}} \mathbf{1} \left( y_{i}\neq c \right) \  \left( \mathrm{donde} \  c=\hat{y}_{t} \right)\\ &=&\displaystyle \frac{1}{m} \sum_{\left( \mathbf{x}_{i} ,y_{i} \right) \in \mathcal{D}_{t}} \mathbf{1} \left( y_{i}\neq f\left( \mathbf{x}_{i} \right) \right)\\ &=&\hat{E}_{\mathrm{entren}} \left( f,\mathcal{D} \right)\end{array}$$
<p style="text-align: right;">$(6.10)$</p>

Donde $\tilde{f}$ es el conjunto de todos los nodos terminales en $f$. Por lo tanto, la aproximación de la ecuación (6.6) mediante estimaciones locales de probabilidad tomadas de cada subespacio $\mathcal{X}_{t}$ es consistente. Luego, **la regla de asignación (6.9) minimiza el error de entrenamiento del modelo en lugar de su error medio de generalización**.

Una propiedad importante de la regla de asignación (6.9) es que cuanto más separamos los nodos intermedios *de cualquier forma* en un árbol de decisión, más pequeño será el error de entrenamiento $\hat{E}_{\mathrm{entren}} \left( f,\mathcal{D} \right)$. Esto significa que **los árboles de clasificación (modelos de tipo árbol que resuelven problemas de clasificación) son muy propensos al overfitting**. Este importante detalle es algo que revisaremos en profundidad más adelante.

Los resultados anteriores nos permiten establecer la siguiente proposición.

**<font color='teal'>Proposición 6.1:</font>** *Sea $f$ un modelo de tipo árbol de decisión, el cual se entrena sobre un conjunto de entrenamiento $\mathcal{D} =\left\{ \left( \mathbf{X} ,\mathbf{y} \right) :\mathbf{X} \in \mathbb{R}^{m\times n} \wedge \mathbf{y} \in \mathbb{R}^{m} \right\}$, donde $\mathbf{y}$ es una vector que contiene únicamente valores iguales a $0$ o $1$. Sea $\tilde{f}$ el conjunto conformado por todos los nodos terminales (u hojas) de $f$. Para cualquier split $s_{t}$ no vacío en un nodo terminal $t\in \tilde{f}$, que produce los nodos hijos $t_{L}$ y $t_{R}$, resultando en un nuevo árbol $f^{\ast}$, donde $\hat{y}_{t_{L}}$ e $\hat{y}_{t_{R}}$ se asignan conforme la regla (6.9), se tiene que*

$$\hat{E}_{\mathrm{entren}} \left( f,\mathcal{D} \right) \geq \hat{E}_{\mathrm{entren}} \left( f^{\prime},\mathcal{D} \right)$$
<p style="text-align: right;">$(6.11)$</p>

*Donde la igualdad se cumple si y sólo si $\hat{y}_{t} =\hat{y}_{t_{L}} =\hat{y}_{t_{R}}$*. □

**<font color='teal'>Demostración 6.1:</font>** La demostración resulta de comparar las expresiones del error de entrenamiento en cada caso y las probabilidades/estimaciones correspondientes:

$$\begin{array}{lrcl}&\hat{E}_{\mathrm{entren}} \left( f,\mathcal{D} \right)&\geq&\displaystyle \hat{E}_{\mathrm{entren}} \left( f^{\prime},\mathcal{D} \right)\\ \Longleftrightarrow&\displaystyle \sum_{t\in \tilde{f}} p\left( t \right) \left( 1-p\left( \hat{y}_{t} |t \right) \right)&\geq&\displaystyle \sum_{t\in \tilde{f}^{\prime}} p\left( t \right) \left( 1-p\left( \hat{y}_{t} |t \right) \right)\\ \Longleftrightarrow&\displaystyle p\left( t \right) \left( 1-p\left( \hat{y}_{t} |t \right) \right)&\geq&\displaystyle p\left( t_{L} \right) \left( 1-p\left( \hat{y}_{t_{L}} |t_{L} \right) \right) +p\left( t_{R} \right) \left( 1-p\left( \hat{y}_{t_{R}} |t_{R} \right) \right)\\ \Longleftrightarrow&\displaystyle \frac{m_{t}}{m} \left( 1-\max_{c\in \mathcal{Y}} \left\{ p\left( c|t \right) \right\} \right)&\geq&\displaystyle \frac{m_{t_{L}}}{m} \left( 1-\max_{c\in \mathcal{Y}} \left\{ p\left( c|t_{L} \right) \right\} \right) +\frac{m_{t_{R}}}{m} \left( 1-\max_{c\in \mathcal{Y}} \left\{ p\left( c|t_{R} \right) \right\} \right)\\ \Longleftrightarrow&\displaystyle \frac{m_{t}}{m} \left( 1-\max_{c\in \mathcal{Y}} \left\{ \frac{n_{c}\left( t \right)}{m_{t}} \right\} \right)&\geq&\displaystyle \frac{m_{t_{L}}}{m} \left( 1-\max_{c\in \mathcal{Y}} \left\{ \frac{n_{c}\left( t_{L} \right)}{m_{t_{L}}} \right\} \right) +\frac{m_{t_{R}}}{m} \left( 1-\max_{c\in \mathcal{Y}} \left\{ \frac{n_{c}\left( t_{R} \right)}{m_{t_{R}}} \right\} \right)\\ \Longleftrightarrow&\displaystyle \frac{m_{t}}{m} \left( 1-\frac{1}{m_{t}} \max_{c\in \mathcal{Y}} \left\{ n_{c}\left( t \right) \right\} \right)&\geq&\displaystyle \frac{m_{t_{L}}}{m} \left( 1-\frac{1}{m_{t_{L}}} \max_{c\in \mathcal{Y}} \left\{ n_{c}\left( t_{L} \right) \right\} \right) +\frac{m_{t_{R}}}{m} \left( 1-\frac{1}{m_{t_{R}}} \max_{c\in \mathcal{Y}} \left\{ n_{c}\left( t_{R} \right) \right\} \right)\\ \Longleftrightarrow&\displaystyle m_{t}-\max_{c\in \mathcal{Y}} \left\{ n_{c}\left( t \right) \right\}&\geq&\displaystyle \left( m_{t_{L}}-\max_{c\in \mathcal{Y}} \left\{ n_{c}\left( t_{L} \right) \right\} \right) +\left( m_{t_{R}}-\max_{c\in \mathcal{Y}} \left\{ n_{c}\left( t_{R} \right) \right\} \right) \  \left( \mathrm{notemos\  que} \  m_{t}=m_{t_{L}}+m_{t_{R}} \right)\\ \Longleftrightarrow&\displaystyle \max_{c\in \mathcal{Y}} \left\{ n_{c}\left( t \right) \right\}&\leq&\displaystyle \max_{c\in \mathcal{Y}} \left\{ n_{c}\left( t_{L} \right) \right\} +\max_{c\in \mathcal{Y}} \left\{ n_{c}\left( t_{R} \right) \right\}\\ \Longleftrightarrow&\displaystyle \max_{c\in \mathcal{Y}} \left\{ n_{c}\left( t_{L} \right) +n_{c}\left( t_{R} \right) \right\}&\leq&\displaystyle \max_{c\in \mathcal{Y}} \left\{ n_{c}\left( t_{L} \right) \right\} +\max_{c\in \mathcal{Y}} \left\{ n_{c}\left( t_{R} \right) \right\}\end{array}$$
<p style="text-align: right;">$(6.12)$</p>

La última línea de la expresión (6.12) es cierta, puesto que $\max_{c\in \mathcal{Y}} \left\{ m_{c\left( t_{L} \right)} \right\}$ es necesariamente mayor o igual que cualquier $m_{c\left( t_{L} \right)}$, y análogamente para $t_{R}$. La igualdad se cumple, naturalmente, si la mayor proporción de instancias pertenecientes a una clase es exactamente la misma en $t$, $t_{L}$ y $t_{R}$, lo que implica $\hat{y}_{t} =\hat{y}_{t_{L}} =\hat{y}_{t_{R}}$. □

Como corolario de la proposición (6.1), podemos establecer que el error de estimación sobre los datos de entrenamiento del árbol de decisión será mínimo cuando sus nodos terminales ya no puedan ser divididos por un *split*. En particular, será igual a cero si el árbol puede *desarrollarse completamente*; esto es, si todos sus nodos terminales pueden dividirse de manera tal que éstos contengan exactamente un único elemento del conjunto de entrenamiento $\mathcal{D}$.

### Problema de regresión.
Cuando el problema de interés representado por el conjunto de entrenamiento $\mathcal{D} =\left\{ \left( \mathbf{X} ,\mathbf{y} \right) :\mathbf{X} \in \mathbb{R}^{m\times n} \wedge \mathbf{y} \in \mathbb{R}^{m} \right\}$ es de regresión, la función de costo $L$ para un modelo $f$ que se entrena sobre $\mathcal{D}$ suele expresar, de alguna manera, la diferencia existente entre las predicciones hechas por el modelo y el valor real de las instancias $y_{1},...,y_{m}$. Una elección popular es el error cuadrático medio, definido como

$$L\left( f,\mathcal{D} \right) =\left\Vert \mathbf{y} -f\left( \mathbf{X} \right) \right\Vert^{2}$$
<p style="text-align: right;">$(6.13)$</p>

Cuando $f$ es un modelo de tipo árbol, cada nodo términal hará una predicción constante para todas las instancias que “caigan” en dicho nodo. Al igual que en el caso de un problema de clasificación, veremos cómo asignar dicho valor constante y cómo estimar el error resultante de dicha asignación.

Sea pues $t$ un nodo terminal del modelo de tipo árbol $f$, y denotemos por $\mathcal{X}_{t}$ el subespacio de instancias de $\mathcal{D}$ que se propagan por el árbol hasta $t$. Sea $m_{t}$ el número de instancias de $\mathcal{X}_{t}$. Como en el caso de los problemas de clasificación con pérdida binaria, donde escogimos la clase más frecuente de las instancias en $\mathcal{X}_{t}$ para minimizar el error local en $t$, ahora buscamos la constante $\hat{y}_{t}$ que minimice la suma de los errores al cuadrado en el nodo $t$. Por lo tanto, cada nodo $t$ del árbol puede describirse por un problema de optimización local (no restringido) tal que

$$\begin{array}{lll}\hat{y}_{t}^{\ast}&=&\displaystyle \underset{\hat{y}_{t} \in \mathbb{R}}{\mathrm{argmin}} \left( \sum_{\left( \mathbf{x}_{i} ,y_{i} \right) \in \mathcal{D}_{t}} \left( y_{i}-\hat{y}_{t} \right)^{2} \right)\\ &=&\displaystyle \frac{1}{m_{t}} \sum_{\left( \mathbf{x}_{i} ,y_{i} \right) \in \mathcal{D}_{t}} y_{i}\end{array}$$
<p style="text-align: right;">$(6.14)$</p>

Donde $\mathcal{D}_{t} =\left\{ \left( \mathbf{x}_{i} ,y_{i} \right) |\mathbf{x}_{i} \in \mathcal{X}_{t} \wedge i=1,...,m_{t} \right\}$. De esta forma, cada nodo terminal $t$ "predice" el promedio de los valores de salida $\left\{ y_{i} \right\}$ que caen en él.

De la misma forma que en el caso de clasificación, si usamos $p(t)=\frac{m_{t}}{m}$ como estimador de la probabilidad $P\left( \mathbf{x}_{i} \in \mathcal{X}_{t} \right)$, y aproximamos el error global de generalización usando tal estimación, obtenemos

$$\begin{array}{lll}E\left( f,\mathcal{D} \right)&=&\displaystyle \sum_{t\in \tilde{f}} \underbrace{p\left( t \right)}_{=\frac{m_{t}}{m}} \cdot \underbrace{\frac{1}{m_{t}} \sum_{\left( \mathbf{x}_{i} ,y_{i} \right) \in \mathcal{D}_{t}} \left( y_{i}-\hat{y}_{t} \right)^{2}}_{\mathrm{promedio\  local\  de\  errores\  al\  cuadrado}}\\ &=&\displaystyle \sum_{t\in \tilde{f}} \frac{m_{t}}{m} \frac{1}{m_{t}} \sum_{\left( \mathbf{x}_{i} ,y_{i} \right) \in \mathcal{D}_{t}} \left( y_{i}-\hat{y}_{t} \right)^{2}\\ &=&\displaystyle \frac{1}{m} \sum_{t\in \tilde{f}} \sum_{\left( \mathbf{x}_{i} ,y_{i} \right) \in \mathcal{D}_{t}} \left( y_{i}-\hat{y}_{t} \right)^{2}\\ &=&\displaystyle \frac{1}{m} \sum_{i=1}^{m} \left( y_{i}-f\left( \mathbf{x}_{i} \right) \right)^{2}\\ &=&\hat{E}_{\mathrm{entren}} \left( f,\mathcal{D} \right)\end{array}$$
<p style="text-align: right;">$(6.15)$</p>

Donde $\tilde{f}$ es el conjunto de todos los nodos terminales del modelo de tipo árbol $f$. Llegamos pues a la misma conclusión que en el caso de los problemas de clasificación: Al reemplazar las probabilidades reales $P\left( \mathbf{x}_{i} \in \mathcal{X}_{t} \right)$ por $p(t)=\frac{m_{t}}{m}$ y la media condicional $\mathrm{E}_{X,Y|t} [\left( y_{i}-\hat{y}_{t} \right)^{2} ]$ ($i=1,...,m$) por la media muestral de $\left( y_{i}-\hat{y}_{t} \right)^{2}$ en cada hoja, concluimos que el error cuadrático esperado (bajo la distribución *empírica* provista por los datos) coincide exactamente con la suma de cuadrados de entrenamiento, dividida por el número $m$ de instancias en $\mathcal{D}$. Esto demuestra que el “error de generalización” (o de predicción) estimado con los datos de entrenamiento es exactamente el mismo que el error de entrenamiento (empírico) del modelo de tipo árbol $f$. Asimismo, la regla de asignación (6.14) permite establecer que, cuanto más separemos los nodos intermedios *de cualquier forma* en un árbol de decisión, más pequeño será el error de entrenamiento $\hat{E}_{\mathrm{entren}} \left( f,\mathcal{D} \right)$. Por lo tanto, los árboles de regresión son igualmente propensos al overfitting que los árboles de clasificación.

Conforme lo anterior, podemos establecer el siguiente resultado.

**<font color='teal'>Proporsición 6.2:</font>** *Sea $f$ un modelo de tipo árbol de decisión, el cual se entrena sobre un conjunto de entrenamiento $\mathcal{D} =\left\{ \left( \mathbf{X} ,\mathbf{y} \right) :\mathbf{X} \in \mathbb{R}^{m\times n} \wedge \mathbf{y} \in \mathbb{R}^{m} \right\}$. Sea $\tilde{f}$ el conjunto conformado por todos los nodos terminales (u hojas) de $f$. Para cualquier split $s_{t}$ no vacío en un nodo terminal $t\in \tilde{f}$, que produce los nodos hijos $t_{L}$ y $t_{R}$, resultando en un nuevo árbol $f^{\ast}$, donde $\hat{y}_{t_{L}}$ e $\hat{y}_{t_{R}}$ se asignan conforme la regla (6.14), se tiene que*

$$\hat{E}_{\mathrm{entren}} \left( f,\mathcal{D} \right) \geq \hat{E}_{\mathrm{entren}} \left( f^{\prime},\mathcal{D} \right)$$
<p style="text-align: right;">$(6.16)$</p>

*Donde la igualdad se cumple si y sólo si $\hat{y}_{t} =\hat{y}_{t_{L}} =\hat{y}_{t_{R}}$*. □

**<font color='teal'>Demostración 6.2:</font>** La proposición (6.2) es exactamente igual a la proposición (6.1), lo que pone de manifiesto que este resultado es válido para todo modelo de tipo árbol, sin importar el problema de interés (clasificación o regresión). Para probar (6.16), bastará con reemplazar cada error con las correspondientes expresiones que aproximan sus componentes, de manera que

$$\begin{array}{rrcl}&\hat{E}_{\mathrm{entren}} \left( f,\mathcal{D} \right)&\geq&\hat{E}_{\mathrm{entren}} \left( f^{\prime},\mathcal{D} \right)\\ \Longleftrightarrow&\displaystyle \sum_{t\in \tilde{f}} p\left( t \right) \left( \frac{1}{m_{t}} \sum_{\left( \mathbf{x}_{i} ,y_{i} \right) \in \mathcal{D}_{t}} \left( y_{i}-\hat{y}_{t} \right)^{2} \right)&\geq&\displaystyle \sum_{t\in \tilde{f^{\prime}}} p\left( t \right) \left( \frac{1}{m_{t}} \sum_{\left( \mathbf{x}_{i} ,y_{i} \right) \in \mathcal{D}_{t}} \left( y_{i}-\hat{y}_{t} \right)^{2} \right)\\ \Longleftrightarrow&\displaystyle \sum_{\left( \mathbf{x}_{i} ,y_{i} \right) \in \mathcal{D}_{t}} \left( y_{i}-\hat{y}_{t} \right)^{2}&\geq&\displaystyle \sum_{\left( \mathbf{x}_{p} ,y_{p} \right) \in \mathcal{D}_{t_{L}}} \left( y_{p}-\hat{y}_{t_{L}} \right)^{2} +\sum_{\left( \mathbf{x}_{q} ,y_{q} \right) \in \mathcal{D}_{t_{R}}} \left( y_{q}-\hat{y}_{t_{R}} \right)^{2}\\ \Longleftrightarrow&\displaystyle m_{t}\hat{y}_{t}^{2}&\leq&\displaystyle m_{t_{L}}\hat{y}_{t_{L}}^{2} +m_{t_{R}}\hat{y}_{t_{R}}^{2}\\ \Longleftrightarrow&\displaystyle \frac{1}{m_{t}} \left( \sum_{\left( \mathbf{x}_{i} ,y_{i} \right) \in \mathcal{D}_{t}} y_{i} \right)^{2}&\leq&\displaystyle \frac{1}{m_{t_{L}}} \left( \sum_{\left( \mathbf{x}_{p} ,y_{p} \right) \in \mathcal{D}_{t_{L}}} y_{p} \right)^{2} +\frac{1}{m_{t_{R}}} \left( \sum_{\left( \mathbf{x}_{q} ,y_{q} \right) \in \mathcal{D}_{t}} y_{q} \right)^{2}\end{array}$$
<p style="text-align: right;">$(6.17)$</p>

Pongamos $s\left( t \right) =\sum_{\left( \mathbf{x}_{i} ,y_{i} \right) \in \mathcal{D}_{t}} y_{i}=m_{t}\hat{y}_{t}$. De esta manera, $s(t)=s(t_{L})+s(t_{R})$. Por lo tanto,

$$\begin{array}{lrcl}\Longleftrightarrow&\displaystyle \frac{\left( s\left( t \right) \right)^{2}}{m_{t}}&\leq&\displaystyle \frac{\left( s\left( t_{L} \right) \right)^{2}}{m_{t_{L}}} +\frac{\left( s\left( t_{R} \right) \right)^{2}}{m_{t_{R}}}\\ \Longleftrightarrow&\displaystyle \frac{\left( s\left( t_{L} \right) +s\left( t_{R} \right) \right)^{2}}{m_{t_{L}}+m_{t_{R}}}&\leq&\displaystyle \frac{\left( s\left( t_{L} \right) \right)^{2}}{m_{t_{L}}} +\frac{\left( s\left( t_{R} \right) \right)^{2}}{m_{t_{R}}}\\ \Longleftrightarrow&\displaystyle \frac{\left( s\left( t_{L} \right) m_{t_{R}}-s\left( t_{R} \right) m_{t_{L}} \right)^{2}}{m_{t_{L}}m_{t_{R}}\left( m_{t_{L}}+m_{t_{R}} \right)}&\geq&0\end{array}$$
<p style="text-align: right;">$(6.18)$</p>

La última línea de la expresión (6.18) es cierta, puesto que el denominador $m_{t_{L}}m_{t_{R}}\left( m_{t_{L}}+m_{t_{R}} \right)$ es estrictamente positivo, mientras que el numerador $\left( s\left( t_{L} \right) m_{t_{R}}-s\left( t_{R} \right) m_{t_{L}} \right)^{2}$ es no negativo por propiedades de la función cuadrática. La igualdad se cumple si y sólo si $s\left( t_{L} \right) m_{t_{R}}=s\left( t_{R} \right) m_{t_{L}}$, lo que se cumple siempre que $\hat{y}_{t_{L}} =\hat{y}_{t_{R}}$. □

Nuevamente, como corolario de la proposición (6.2), podemos establecer que el error de entrenamiento del modelo de tipo árbol $f$ será mínimo cuando ya no sea posible dividir sus nodos terminales, siendo igual a cero cuando exactamente una instancia del conjunto de entrenamiento $\mathcal{D}$ esté contenida en dichos nodos.